# ResNet-50 Learning

Time to put our dataset to use! We'll use pytorch-lightning as a trainer!

If you'd like to see, how well does the backbone work compared to using transfer with the main article ResNet-50, see ResNet-50 Backbone Testing notebook.

### Constants and imports

In [1]:
import os
import torch
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from lightning.pytorch import Trainer, seed_everything, LightningModule
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.io import read_image
from torchvision.transforms import v2

METADATA_DIRECTORY = 'metadata'
OUTPUT_FILE = os.path.join(METADATA_DIRECTORY, 'dataset.csv')
CHECKPOINTS_DIRECTORY = 'checkpoints'
TEST_SIZE = 0.3
RANDOM_STATE = 42
BATCH_SIZE = 32
EPOCH_COUNT = 30
PATIENCE = 20

### Preparing directories

In [2]:
if not os.path.exists(CHECKPOINTS_DIRECTORY):
    os.makedirs(CHECKPOINTS_DIRECTORY)

### Loading the dataset

In [3]:
df = pd.read_csv(OUTPUT_FILE)

classes = sorted(pd.unique(df['class']))

/tmp/ipykernel_757734/3190094445.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(OUTPUT_FILE)


In [4]:
len(classes)

70

In [5]:
len(df.index)

133129

### Splitting the dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['path'], df['class'], test_size=TEST_SIZE, random_state=RANDOM_STATE)

### Define transformations

In [7]:
train_transforms = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.2),
    v2.RandomRotation(20),
    v2.Resize(size=(224, 224), antialias=True),
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05)
])

test_transforms = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
])

### Creating dataloaders

In [8]:
class MuseumCC0Dataset(Dataset):
    def __init__(self, Xs, ys, transform=None):
        self.Xs = list(Xs)
        self.ys = torch.tensor([classes.index(y) for y in ys])
        self.transform = transform
        
    def __len__(self):
        return len(self.Xs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = read_image(self.Xs[idx])

        image = image.float() / 255

        if image.size()[0] == 1:
            image = image.repeat(3, 1, 1)

        if image.size()[0] == 4:
            image = image[1:4]
        
        if self.transform:
            image = self.transform(image)

        return image, self.ys[idx]

train_dataset = MuseumCC0Dataset(X_train, y_train, train_transforms)
test_dataset = MuseumCC0Dataset(X_test, y_test, test_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=5)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=5)

### Creating the trainer

In [9]:
def create_model():
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=None)
    model.fc = torch.nn.Linear(model.fc.in_features, len(classes))
    return model
                               
class MuseumCC0Module(LightningModule):
    def __init__(self):
        super().__init__()

        self.model = create_model()
        self.loss_module = torch.nn.CrossEntropyLoss()

    def forward(self, imgs):
        return self.model(imgs)

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.1)   

    def training_step(self, batch, batch_idx):
        images, labels = batch

        preds = self.model(images)
        
        loss = self.loss_module(preds, labels)

        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log("train_acc", acc, on_step=True, on_epoch=True)
        self.log("train_loss", loss)
        
        return loss 

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        preds = self.model(images)

        loss = self.loss_module(preds, labels)

        acc = (labels == preds.argmax(dim=-1)).float().mean()

        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)

trainer = Trainer(default_root_dir=CHECKPOINTS_DIRECTORY, callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=PATIENCE)], max_epochs=EPOCH_COUNT)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/macron/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


### Fitting the model

In [ ]:
torch.set_float32_matmul_precision('high')

model = MuseumCC0Module()

trainer.fit(model, train_dataloader, test_dataloader)

Using cache found in /home/macron/.cache/torch/hub/pytorch_vision_v0.10.0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | ResNet           | 23.7 M
1 | loss_module | CrossEntropyLoss | 0     
-------------------------------------------------
23.7 M    Trainable params
0         Non-trainable params
23.7 M    Total params
94.606    Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Corrupt JPEG data: 32765 extraneous bytes before marker 0xd9


Validation: |                                             | 0/? [00:00<?, ?it/s]

Corrupt JPEG data: 32762 extraneous bytes before marker 0xd9
Corrupt JPEG data: 32765 extraneous bytes before marker 0xd9
